In [1]:
import requests
import os
from functools import cached_property
from collections import defaultdict
import json
from dotenv import load_dotenv
load_dotenv() # includes the CHALLONGE_API_KEY variable

True

# Define code to download and parse Wingspan tournaments from Challonge

In [2]:
def challonge_get(url, retries=3):
    try:
        return requests.get(url, {
            'api_key': os.environ['CHALLONGE_API_KEY']
        }, headers={
            'Content-Type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
        }).json()
    except:
        print(f"failed to get {url}")
        if retries > 0:
            return challonge_get(url, retries-1)

valid_owners = {'Bargoff',
 'ChopYouUp',
 'ElsieGlen',
 'FlanHigh',
 'FloatingLakes',
 'Gtrudat',
 'Lone_Eider7',
 'ScaredofBirds',
 'deVisme'}


class Game:
    def __init__(self, scores, players):
        self.scores = scores
        self.players = players

    @property
    def score_dict(self):
        return { player: score for player, score in zip(self.players, self.scores) }
        
class Match:
    def __init__(self, attrs):
        self.attrs = attrs

    @property
    def scores(self):
        try:
            return [[int(s) for s in scores.split('-')]
                    for scores in self.attrs['scores_csv'].split(',')]
        except:
            import pdb; pdb.set_trace()

    @property
    def players(self): return [self.attrs['player1_id'], self.attrs['player2_id']]

    @property
    def games(self):
        return [Game(scores, self.players) for scores in self.scores]
        
class Tournament:
    @classmethod
    def search(cls, max=1000):
        """Use the challonge search feature to get a list of Wingspan tournaments"""
        url = f"https://challonge.com/search/tournaments.json?filters%5Bgame_id%5D=215240&per={max}"
        results = challonge_get(url)['collection']
        results = [r for r in results if r['owner'] in valid_owners]
        tourneys = [cls.by_url(r['link']) for r in results]
        tourneys = [t for t in tourneys if not t.name.lower().startswith('test')]
        return tourneys
    
    @classmethod
    def by_url(cls, url):
        id = challonge_get(f"{url}.json")['tournament']['id']
        return cls.by_id(id)

    @classmethod
    def by_id(cls, id):
        return cls(challonge_get(f"https://api.challonge.com/v1/tournaments/{id}.json")['tournament'])

    def __init__(self, attrs):
        self.attrs = attrs

    @property
    def name(self):
        name = self.attrs['name']
        name = name.strip()
        return name

    @property
    def id(self): return self.attrs['id']

    @property
    def url(self): return f"https://challonge.com/{self.attrs['url']}"

    @property
    def date(self): return self.attrs['started_at']
        
    @cached_property
    def participants(self):
        return [p['participant'] for p in challonge_get(f"https://api.challonge.com/v1/tournaments/{self.id}/participants.json")]

    @cached_property
    def participants_by_id(self):
        by_id = {}
        for p in self.participants:
            by_id[p['id']] = p['name']
            if 'group_player_ids' in p:
                for id in p['group_player_ids']:
                    assert id not in by_id
                    by_id[id] = p['name']
        return by_id
            
    @cached_property
    def matches(self):
        return [
            Match(m['match'])
            for m in challonge_get(f"https://api.challonge.com/v1/tournaments/{self.id}/matches.json")
            if m['match']['state'] == 'complete'
        ]

    @property
    def games(self):
        return [game for match in self.matches for game in match.games]

    @cached_property
    def scores_by_participant(self):
        scores = defaultdict(list)
        for g in self.games:
            for pid, score in g.score_dict.items():
                scores[self.participants_by_id[pid]].append(score)
        return scores

    @cached_property
    def as_json(self):
        return {
              'id': self.id,
              'name': self.name,
              'url': self.url,
              'date': self.date,
              'scores': [{
                  self.participants_by_id[game.players[0]]: game.scores[0],
                  self.participants_by_id[game.players[1]]: game.scores[1] 
              } for game in self.games]
        }
        

In [3]:
t = Tournament(
    challonge_get("https://api.challonge.com/v1/tournaments/13589704.json")['tournament'])
t.name

'Wingspan World Cup Regionals - Asia'

In [4]:
t.as_json

{'id': 13589704,
 'name': 'Wingspan World Cup Regionals - Asia',
 'url': 'https://challonge.com/7ldxp0zx',
 'date': '2023-10-14T16:34:11.774+08:00',
 'scores': [{'mike316': 97, 'thebangzats': 86},
  {'MrSaber': 85, 'Zeemokung': 87},
  {'Sim17': 94, 'otsukuuyan': 85},
  {'WillyPizza': 87, 'Salmonskinroll': 93},
  {'smelly_cat (smellysmellycat)': 85, 'gdhar67 (GasLighter)': 90},
  {'Salmonskinroll': 89, 'smelly_cat (smellysmellycat)': 93},
  {'otsukuuyan': 65, 'WillyPizza': 104},
  {'Zeemokung': 77, 'Sim17': 114},
  {'thebangzats': 93, 'MrSaber': 95},
  {'Arceusisgood': 104, 'mike316': 85},
  {'MrSaber': 108, 'Arceusisgood': 110},
  {'Sim17': 86, 'thebangzats': 101},
  {'WillyPizza': 80, 'Zeemokung': 90},
  {'smelly_cat (smellysmellycat)': 74, 'otsukuuyan': 83},
  {'gdhar67 (GasLighter)': 101, 'Salmonskinroll': 85},
  {'otsukuuyan': 117, 'gdhar67 (GasLighter)': 97},
  {'Zeemokung': 89, 'smelly_cat (smellysmellycat)': 83},
  {'thebangzats': 99, 'WillyPizza': 96},
  {'Arceusisgood': 93, 'S

# Fetch tournaments

In [5]:
tourneys = Tournament.search()

In [6]:
tourney_json = []
bad_tourneys = []
for t in tourneys:
    try:
        tourney_json.append(t.as_json)
    except:
        bad_tourneys.append(t)
tourney_json

[{'id': 13718089,
  'name': 'Xenopsaris X-Factor',
  'url': 'https://challonge.com/wtds_xx',
  'date': None,
  'scores': [{'gaussiancurve': 102, 'spocpes': 75},
   {'gaussiancurve': 103, 'spocpes': 71},
   {'gaussiancurve': 0, 'spocpes': 0},
   {'ScaredofBirds': 109, 'DMonkey225': 54},
   {'ScaredofBirds': 83, 'DMonkey225': 56},
   {'ScaredofBirds': 0, 'DMonkey225': 0},
   {'GraDog22': 96, 'Huxley3000': 84},
   {'GraDog22': 86, 'Huxley3000': 102},
   {'GraDog22': 101, 'Huxley3000': 113},
   {'Merisabear': 87, 'BlackSpice28': 108},
   {'Merisabear': 75, 'BlackSpice28': 93},
   {'Merisabear': 0, 'BlackSpice28': 0},
   {'foxjoke': 117, 'WillyPizza': 92},
   {'foxjoke': 88, 'WillyPizza': 113},
   {'foxjoke': 108, 'WillyPizza': 122},
   {'Shepherd0': 104, 'SegalLikeTheBird': 100},
   {'Shepherd0': 95, 'SegalLikeTheBird': 101},
   {'Shepherd0': 101, 'SegalLikeTheBird': 96},
   {'MrSaber': 80, 'cgjunior': 87},
   {'MrSaber': 101, 'cgjunior': 98},
   {'MrSaber': 95, 'cgjunior': 105},
   {'_aqu

# Normalize user names (there are a lot of near-duplicates :/)

In [7]:
users = set()

for t in tourney_json:
    for scores in t['scores']:
        for user, _score in scores.items():
            users.add(user)

users = list(sorted(list(users)))

In [8]:
import re

def normalize_name(name):
    name = name.lower().split('(')[0].split(' -')[0].split('\t')[0].split(' |')[0].split('#')[0]
    name = name.replace("_", " ")
    name = name.replace("ign: ", "")
    name = name.replace(" ", "")
    #name = re.sub(r"(^[a-z])", "", name)
    name = name.strip()
    return name

normalized_user_names = { name: normalize_name(name) for name in users }

In [9]:
from collections import defaultdict

user_counts = defaultdict(int)

for t in tourney_json:
    for scores in t['scores']:
        for user, _score in scores.items():
            user_counts[normalized_user_names[user]] += 1

In [10]:
from difflib import get_close_matches
import numpy as np

substitutions = {}

for name in set(user_counts.keys()):
    other_names = [n for n in set(user_counts.keys()) if n != name]
    matches = get_close_matches(name, other_names, cutoff=0.75)

    if matches:
        names = [name] + matches
        counts = [user_counts[n] for n in names]
        idx = np.argmax(counts)
        for n in names:
            if n != names[idx]:
                substitutions[n] = names[idx]

substitutions

{'elsapoguapobot': 'elsapoguapo',
 'tay-ray': 'tayray',
 'taykay': 'tayray',
 'ethmah': 'ethmah01',
 'chuck': 'chuckaus',
 'danieleku': 'daniel',
 'ltrudat': 'gtrudat',
 'ronster': 'ronster7',
 'iwinatlife': 'iwinatlife42',
 'jesseeeganush': 'jesseeekah',
 'thecomedian91': 'thec0median',
 'thec0median91': 'thec0median',
 'erin': 'erik',
 'irek': 'ireku',
 'team20': 'team23',
 'white-backedwoodpecker': 'blackwoodpecker',
 'sleephead': 'sleepyhead',
 'wingspanner2': 'wingsplain',
 'rantaro31194': 'rantaro311',
 'loneeider': 'loneeider7',
 'nedmund13': 'nedmund',
 'nedmud13': 'nedmund',
 'spartanfan13': 'spartafnfan13',
 'spartafnfan13+wayward': 'spartafnfan13',
 'puffin420': 'puffin',
 'yippecahier': 'yippeecahier',
 'stormchaser20': 'stormchaser',
 'rokb': 'rob',
 'robin': 'rob',
 'stuart': 'stuartgmt9',
 'groovenbeast': 'groovenstein',
 'kellakel': 'kellakel93',
 'erikajoycelovesducks': 'kylelovesducks',
 'ign:camhanoi': 'camhanoi',
 'at-stkampfläufer': 'at-stkampläufer',
 'eurasianspa

In [11]:
substitutions2 = {'sleephead': 'sleepyhead',
 'tay-ray': 'tayray',
 #'taykay': 'tayray',
# 'rokb': 'rob',
# 'robin': 'rob',
# 'dagger': 'digger',
 'żyraf🦒': 'żyraf',
 'zyraf': 'żyraf',
 'thecomedian91': 'thec0median',
 'thec0median91': 'thec0median',
 'irek': 'ireku',
 'no-m': 'no~m',
 'nom': 'no~m',
# 'wingsplaingaming': 'wingspanner',
# 'bluethroat': 'bluethroat4life',
 'wingspanner2': 'wingspanner',
 #'eurasiannuthatch': 'corsicannuthatch',
 #'corsicannuthatch': 'eurasiannuthatch',
 #'team23': 'team20',
 #'sandro': 'andreko',
 'jeasthebeast': 'jeastthebeast',
 #'jeastthebeast+malue': 'jeastthebeast',
# 'eurasianhobby': 'eurasianjay',
# 'ltrudat': 'gtrudat',
 'elsapoguapobot': 'elsapoguapo',
 'dozo': 'dozi',
 'theflash04': 'theflash',
 #'scaredofdiamonds': 'scaredofbirds',
 #'wingsplain': 'wingspanner',
 'seechristy': 'seechristine',
 #'commonmoorhen': 'commonraven',
 #'eurasianjay': 'eurasianhobby',
 #'spartafnfan13+wayward': 'spartanfan13',
 'stormchaser20': 'stormchaser',
 #'merisabear': 'mrsaber',
 'ronster': 'ronster7',
 #'mtrooster': 'ronster',
 'ooievaar🇳🇱': 'ooievaar',
 #'avery': 'av0ry',
 'spartafnfan13': 'spartanfan13',
 #'white-backedwoodpecker': 'blackwoodpecker',
 #'groovenbeast': 'groovenstein',
 'falblingius': 'falblinger',
 #'eurasiansparrowhawk': 'eurasiantreesparrow',
 'nedmund13': 'nedmund',
 'nedmud13': 'nedmund',
 'chuck': 'chuckaus',
 'mothertucker': 'mothertuckers',
# 'eurasiannutcracker': 'eurasiannuthatch',
 'yippecahier': 'yippeecahier',
# 'team20': 'team23',
# 'eurasiantreesparrow': 'eurasiansparrowhawk'
                 }

In [12]:
normalized_user_names2 = { k: substitutions2.get(v, v) for k, v in normalized_user_names.items() }

In [13]:
normalized_user_names2

{'.ineslopes': '.ineslopes',
 '47mattbrown (ColonelSanderling)': '47mattbrown',
 '@thedetermin8or': '@thedetermin8or',
 'A Tale of Two Kiwis': 'ataleoftwokiwis',
 'A-Team': 'a-team',
 'A-Team (Salrantol)': 'a-team',
 'A-Team (Salrantol) - USA': 'a-team',
 'A-Team (salrantol)': 'a-team',
 'ACE283': 'ace283',
 'AT-ST Kampfläufer': 'at-stkampfläufer',
 'AT-ST Kampläufer': 'at-stkampläufer',
 'AWingandaPrayer': 'awingandaprayer',
 'AarBee#6728 (AarBee)': 'aarbee',
 'Abiath': 'abiath',
 'Abiath (Sweden)': 'abiath',
 'Abongdong': 'abongdong',
 'Ace283': 'ace283',
 'AceOfHearts (Donald)': 'aceofhearts',
 'Aelfred (burrit0bell)': 'aelfred',
 'Alex (IGN: Zarypov)': 'alex',
 'Alex (IGN: Zarypov) (Germany)': 'alex',
 'Alex (IGN: Zarypov) (Zarypov)': 'alex',
 'Alex (Zarypov)': 'alex',
 'Alex2bach': 'alex2bach',
 'Andreko': 'andreko',
 'AnnaEsquire (RavenousAnna)': 'annaesquire',
 'Aquaka': 'aquaka',
 'Arceusisgood': 'arceusisgood',
 'Arceusisgood (Hong Kong)': 'arceusisgood',
 'Aristo': 'aristo',


In [14]:
names_matching_normalized = defaultdict(list)
for k, v in normalized_user_names2.items():
    names_matching_normalized[v].append(k)
names_matching_normalized

defaultdict(list,
            {'.ineslopes': ['.ineslopes'],
             '47mattbrown': ['47mattbrown (ColonelSanderling)'],
             '@thedetermin8or': ['@thedetermin8or'],
             'ataleoftwokiwis': ['A Tale of Two Kiwis'],
             'a-team': ['A-Team',
              'A-Team (Salrantol)',
              'A-Team (Salrantol) - USA',
              'A-Team (salrantol)'],
             'ace283': ['ACE283', 'Ace283'],
             'at-stkampfläufer': ['AT-ST Kampfläufer'],
             'at-stkampläufer': ['AT-ST Kampläufer'],
             'awingandaprayer': ['AWingandaPrayer'],
             'aarbee': ['AarBee#6728 (AarBee)'],
             'abiath': ['Abiath', 'Abiath (Sweden)'],
             'abongdong': ['Abongdong'],
             'aceofhearts': ['AceOfHearts (Donald)'],
             'aelfred': ['Aelfred (burrit0bell)'],
             'alex': ['Alex (IGN: Zarypov)',
              'Alex (IGN: Zarypov) (Germany)',
              'Alex (IGN: Zarypov) (Zarypov)',
              'Alex

In [15]:
from collections import Counter

overall_counts = Counter([
    user
    for t in tourney_json
    for scores in t['scores']
    for user, _score in scores.items()
])
overall_counts

Counter({'jeastthebeast': 404,
         'Sim17': 380,
         'FloatingLakes': 352,
         'Tayray': 351,
         'ChopYouUp': 325,
         'Groovenstein': 313,
         'SegalLikeTheBird': 299,
         'ethmah01': 296,
         'Flan': 292,
         'JunePaik': 286,
         'Waterman': 285,
         'ScaredofBirds': 283,
         'EpicSaxGuy': 279,
         'motherlove': 273,
         'bushNIT': 236,
         'BalletMage': 235,
         'de Visme': 225,
         'BlackSpice28': 215,
         'Lauris': 215,
         'GracklinOatBrant': 207,
         'SPAKEME': 206,
         'RickCissel': 199,
         'DrGrainslove': 192,
         'Kestrelly': 189,
         'saephir': 189,
         'Tabpub': 180,
         'matthew9890': 176,
         'de Visme (deVisme2)': 171,
         'gtrudat': 163,
         'MichaelTheGrackle': 159,
         'blueTango': 159,
         'ZobiWanKenobi': 158,
         'Stalledbird': 157,
         'Alex (IGN: Zarypov)': 157,
         'ElSapoGuapo': 153,
        

In [16]:
canonical_names = {}

for normalized_name, variants in names_matching_normalized.items():
    counts_by_variant = [overall_counts[v] for v in variants]
    canonical_names[normalized_name] = variants[np.argmax(counts_by_variant)]

canonical_names

{'.ineslopes': '.ineslopes',
 '47mattbrown': '47mattbrown (ColonelSanderling)',
 '@thedetermin8or': '@thedetermin8or',
 'ataleoftwokiwis': 'A Tale of Two Kiwis',
 'a-team': 'A-Team (Salrantol)',
 'ace283': 'Ace283',
 'at-stkampfläufer': 'AT-ST Kampfläufer',
 'at-stkampläufer': 'AT-ST Kampläufer',
 'awingandaprayer': 'AWingandaPrayer',
 'aarbee': 'AarBee#6728 (AarBee)',
 'abiath': 'Abiath',
 'abongdong': 'Abongdong',
 'aceofhearts': 'AceOfHearts (Donald)',
 'aelfred': 'Aelfred (burrit0bell)',
 'alex': 'Alex (IGN: Zarypov)',
 'alex2bach': 'alex2bach',
 'andreko': 'Andreko',
 'annaesquire': 'AnnaEsquire (RavenousAnna)',
 'aquaka': 'Aquaka',
 'arceusisgood': 'Arceusisgood',
 'aristo': 'Aristo',
 'arrisirjoe': 'Arrisirjoe',
 'audouin’sgull': 'Audouin’s Gull',
 'av0ry': 'Av0ry',
 'avery': 'Avery (Aquilagryphon)',
 'averythepigeon': 'AverythePigeon',
 'btsm': 'BTSM (3)',
 'baller4him': 'Baller4Him (same name ig)',
 'balletmage': 'BalletMage',
 'bandyowl': 'BandyOwl',
 'bargoff': 'Bargoff',
 '

In [17]:
t['scores']

[]

In [18]:
normalized_tourneys = []
for t in tourney_json:
    new_tourney = dict(t)
    scores = new_tourney['scores']
    new_scores = []
    for score in scores:
        new_score = {}
        for k, v in score.items():
            new_score[canonical_names[normalized_user_names2[k]]] = v
        new_scores.append(new_score)
    new_tourney['scores'] = new_scores
    normalized_tourneys.append(new_tourney)

In [19]:
normalized_tourneys[-1]['scores']

[]

# Remove tournaments that are not 2-player Wingspan

In [20]:
normalized_tourneys = [t for t in normalized_tourneys if 'Joint Jabiru Joust' not in t['name']
               and 'Tournament Discord Server Image Bird' not in t['name']
             and 'Birds of a Feather' not in t['name']
            and 'Diamond Firetail Foursomes' not in t['name']]

In [21]:
for t in normalized_tourneys:
    t['scores'] = [scores for scores in t['scores'] if min(scores.values()) > 0]

In [22]:
normalized_tourneys

[{'id': 13718089,
  'name': 'Xenopsaris X-Factor',
  'url': 'https://challonge.com/wtds_xx',
  'date': None,
  'scores': [{'gaussiancurve': 102, 'spocpes': 75},
   {'gaussiancurve': 103, 'spocpes': 71},
   {'ScaredofBirds': 109, 'DMonkey225': 54},
   {'ScaredofBirds': 83, 'DMonkey225': 56},
   {'GraDog22 (=IGN)': 96, 'Huxley3000': 84},
   {'GraDog22 (=IGN)': 86, 'Huxley3000': 102},
   {'GraDog22 (=IGN)': 101, 'Huxley3000': 113},
   {'Merisabear': 87, 'BlackSpice28': 108},
   {'Merisabear': 75, 'BlackSpice28': 93},
   {'Foxjoke': 117, 'WillyPizza': 92},
   {'Foxjoke': 88, 'WillyPizza': 113},
   {'Foxjoke': 108, 'WillyPizza': 122},
   {'Shepherd0': 104, 'SegalLikeTheBird': 100},
   {'Shepherd0': 95, 'SegalLikeTheBird': 101},
   {'Shepherd0': 101, 'SegalLikeTheBird': 96},
   {'MrSaber': 80, 'cgjunior': 87},
   {'MrSaber': 101, 'cgjunior': 98},
   {'MrSaber': 95, 'cgjunior': 105},
   {'_aquanaut': 99, 'Maskonur': 90},
   {'_aquanaut': 71, 'Maskonur': 97},
   {'_aquanaut': 72, 'Maskonur': 9

# Save to JSON

In [23]:
with open('wingspan_tournaments.json', 'w') as f:
    json.dump(normalized_tourneys, f, indent=2)

# Fix more names

In [24]:
with open('wingspan_tournaments.json', 'r') as f:
    normalized_tourneys = json.load(f)

In [25]:
extra_substitutions = {
    'FlanSwitch': 'Flan',
    'Birdwatchermobile': 'Birdwatcher123',
    'Mliguori16 (Birdwatcher123)': 'Birdwatcher123',
    'Cap': 'Capitalist111#3362 (Cap)',
    'HAyAsIiI (slowPigeon)': 'slowPigeon',
    'Jjlig2mobile2': 'Jjlig2',
    'No~M l IGN: danaran': 'No~M (danaran)',
    'Rafa oakie (oakie)': 'Rafa (oakie)',
    'Shiny (ShinyEmmy)': 'ShinyEmmy',
    'Typhus / rnicolas (Typhus)': 'Typhus',
    'Wingsplain': 'Wingsplain Gaming (ShadowFox)',
    "oakie6439\t(oakie)": 'oakie',
    "nsegal14 (SegalLikeTheBird)": "SegalLikeTheBird",
    "e8v2000 (ScaredofBirds)": "ScaredofBirds",
    "eefdeaardappel (ooievaar) - The Netherlands": "ooievaar",
    "ooievaar🇳🇱 (ign moustache)": "ooievaar",
    "Johnlim": "Jlim (Johnlim)",
    "HaveAGoodOne": "Kip (ign: HaveAGoodOne)",
    "cedarwax": "maggie (cedarwax)",
    "mellymelyay": "mcdo0530 (mellymelyay)",
    "nardybux (MonkeyWren)": "MonkeyWren",
    "ScottCS (MortalWings)": "MortalWings",
    "Trung (Stalledbird)": "Stalledbird"
}

for t in normalized_tourneys:
    for s in t['scores']:
        to_substitute = [user for user in s.keys() if user in extra_substitutions]
        for user in to_substitute:
            score = s[user]
            del s[user]
            s[extra_substitutions[user]] = score
                
with open('wingspan_tournaments.json', 'w') as f:
    json.dump(normalized_tourneys, f, indent=2)